<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Data Organization


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [73]:
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/OrganizeExploreAndQuery/raw/main/dataFiles/dengue_ok.pkl"
dengue = pd.read_pickle(linkData)
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398943 entries, 0 to 398942
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   departamento  398943 non-null  object        
 1   provincia     398943 non-null  object        
 2   distrito      398943 non-null  object        
 3   ano           398943 non-null  int64         
 4   semana        398943 non-null  int64         
 5   sexo          398943 non-null  object        
 6   edad          398943 non-null  int64         
 7   enfermedad    398943 non-null  category      
 8   year          398931 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(4)
memory usage: 24.7+ MB


In [74]:
dengue.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,ano,semana,edad,year
count,398943.00000,398943.00000,398943.00000,398931.00000
mean,2015.06170,22.61685,29.97476,.5f
min,2000.00000,1.00000,0.00000,.5f
25%,2011.00000,11.00000,15.00000,.5f
50%,2016.00000,19.00000,27.00000,.5f
75%,2020.00000,34.00000,42.00000,.5f
max,2022.00000,53.00000,106.00000,.5f
std,6.14862,14.89333,18.53260,nan


Each row is a person:

In [75]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01


If we wanted to count people, creating a column of ones helps:

In [76]:
dengue=dengue.assign(case=1)
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01,1


Let's start creating _data from these data_!

## Aggregation

In [176]:
indexList=['departamento', 'provincia', 'ano', 'semana','enfermedad']
aggregator={'edad': ['mean'], 'case':['sum']}
ByProvinceWeek_sum_mean=dengue.groupby(indexList,observed=True).agg(aggregator)
ByProvinceWeek_sum_mean

edad case
                                                      mean  sum
departamento provincia  ano  semana enfermedad                 
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES  34.352941   17
                             19     SIN_SEÑALES  29.825000   40
                             20     SIN_SEÑALES  35.172414   58
                             21     SIN_SEÑALES  37.444444   27
                             22     SIN_SEÑALES  30.708333   24
...                                                    ...  ...
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES  30.800000    5
                                    ALARMA       33.500000    2
             PURUS      2020 51     SIN_SEÑALES  46.000000    1
                        2022 28     ALARMA       17.000000    1
                             51     SIN_SEÑALES  43.000000    1

[36422 rows x 2 columns]

Notice:

In [177]:
ByProvinceWeek_sum_mean.columns

MultiIndex([('edad', 'mean'),
            ('case',  'sum')],
           )

We may turn that multi-index structure in a simpler way:

In [178]:
ByProvinceWeek_sum_mean.columns=['meanAge','sumCases']
ByProvinceWeek_sum_mean

meanAge  sumCases
departamento provincia  ano  semana enfermedad                      
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES  34.352941        17
                             19     SIN_SEÑALES  29.825000        40
                             20     SIN_SEÑALES  35.172414        58
                             21     SIN_SEÑALES  37.444444        27
                             22     SIN_SEÑALES  30.708333        24
...                                                    ...       ...
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES  30.800000         5
                                    ALARMA       33.500000         2
             PURUS      2020 51     SIN_SEÑALES  46.000000         1
                        2022 28     ALARMA       17.000000         1
                             51     SIN_SEÑALES  43.000000         1

[36422 rows x 2 columns]

In [179]:
ByProvinceWeek_sum_mean.reset_index(drop=False,inplace=True)

ByProvinceWeek_sum_mean

,departamento,provincia,ano,semana,enfermedad,meanAge,sumCases
0,AMAZONAS,BAGUA,2000,18,SIN_SEÑALES,34.352941,17
1,AMAZONAS,BAGUA,2000,19,SIN_SEÑALES,29.825000,40
2,AMAZONAS,BAGUA,2000,20,SIN_SEÑALES,35.172414,58
3,AMAZONAS,BAGUA,2000,21,SIN_SEÑALES,37.444444,27
4,AMAZONAS,BAGUA,2000,22,SIN_SEÑALES,30.708333,24
...,...,...,...,...,...,...,...
36417,UCAYALI,PADRE ABAD,2022,52,SIN_SEÑALES,30.800000,5
36418,UCAYALI,PADRE ABAD,2022,52,ALARMA,33.500000,2
36419,UCAYALI,PURUS,2020,51,SIN_SEÑALES,46.000000,1
36420,UCAYALI,PURUS,2022,28,ALARMA,17.000000,1


In [181]:
ByProvinceWeek_sum_mean.enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)

In [182]:
ByProvinceWeek_sum_mean.to_pickle('dataFiles/ByProvinceWeek_sum_mean.pkl')

## Reshaping

### From Long to Wide

The object *CasesByWeek* shows the values in one column, and the other column serve as identifier (an index). Let's make a simple wide version (one index):

In [183]:
ByProvinceWeek_sum_mean_Wide=ByProvinceWeek_sum_mean.pivot_table(index=['departamento','provincia','ano','semana'],
                                                                 columns='enfermedad')
ByProvinceWeek_sum_mean_Wide

meanAge                 sumCases  \
enfermedad                          SIN_SEÑALES ALARMA GRAVE SIN_SEÑALES   
departamento provincia  ano  semana                                        
AMAZONAS     BAGUA      2000 18       34.352941    NaN   NaN        17.0   
                             19       29.825000    NaN   NaN        40.0   
                             20       35.172414    NaN   NaN        58.0   
                             21       37.444444    NaN   NaN        27.0   
                             22       30.708333    NaN   NaN        24.0   
...                                         ...    ...   ...         ...   
UCAYALI      PADRE ABAD 2022 51       16.111111   17.0   NaN         9.0   
                             52       30.800000   33.5   NaN         5.0   
             PURUS      2020 51       46.000000    NaN   NaN         1.0   
                        2022 28             NaN   17.0   NaN         NaN   
                             51       43.000000    NaN   NaN         1.0   

                                                  
enfermedad                          ALARMA GRAVE  
departamento provincia  ano  semana               
AMAZONAS     BAGUA      2000 18        NaN   NaN  
                             19        NaN   NaN  
                             20        NaN   NaN  
                             21        NaN   NaN  
                             22        NaN   NaN  
...                                    ...   ...  
UCAYALI      PADRE ABAD 2022 51        1.0   NaN  
                             52        2.0   NaN  
             PURUS      2020 51        NaN   NaN  
                        2022 28        1.0   NaN  
                             51        NaN   NaN  

[27203 rows x 6 columns]

Simpler structure:

In [184]:
ByProvinceWeek_sum_mean_Wide_flatten=ByProvinceWeek_sum_mean.pivot_table(index=['departamento','provincia','ano','semana'],
                            columns='enfermedad').reset_index(drop=False)
ByProvinceWeek_sum_mean_Wide_flatten

departamento   provincia   ano semana     meanAge               \
enfermedad                                       SIN_SEÑALES ALARMA GRAVE   
0              AMAZONAS       BAGUA  2000     18   34.352941    NaN   NaN   
1              AMAZONAS       BAGUA  2000     19   29.825000    NaN   NaN   
2              AMAZONAS       BAGUA  2000     20   35.172414    NaN   NaN   
3              AMAZONAS       BAGUA  2000     21   37.444444    NaN   NaN   
4              AMAZONAS       BAGUA  2000     22   30.708333    NaN   NaN   
...                 ...         ...   ...    ...         ...    ...   ...   
27198           UCAYALI  PADRE ABAD  2022     51   16.111111   17.0   NaN   
27199           UCAYALI  PADRE ABAD  2022     52   30.800000   33.5   NaN   
27200           UCAYALI       PURUS  2020     51   46.000000    NaN   NaN   
27201           UCAYALI       PURUS  2022     28         NaN   17.0   NaN   
27202           UCAYALI       PURUS  2022     51   43.000000    NaN   NaN   

              sumCases               
enfermedad SIN_SEÑALES ALARMA GRAVE  
0                 17.0    NaN   NaN  
1                 40.0    NaN   NaN  
2                 58.0    NaN   NaN  
3                 27.0    NaN   NaN  
4                 24.0    NaN   NaN  
...                ...    ...   ...  
27198              9.0    1.0   NaN  
27199              5.0    2.0   NaN  
27200              1.0    NaN   NaN  
27201              NaN    1.0   NaN  
27202              1.0    NaN   NaN  

[27203 rows x 10 columns]

We have multi index, let's flatten them:

In [185]:
ByProvinceWeek_sum_mean_Wide_flatten.columns

MultiIndex([('departamento',            ''),
            (   'provincia',            ''),
            (         'ano',            ''),
            (      'semana',            ''),
            (     'meanAge', 'SIN_SEÑALES'),
            (     'meanAge',      'ALARMA'),
            (     'meanAge',       'GRAVE'),
            (    'sumCases', 'SIN_SEÑALES'),
            (    'sumCases',      'ALARMA'),
            (    'sumCases',       'GRAVE')],
           names=[None, 'enfermedad'])

In [186]:
ByProvinceWeek_sum_mean_Wide_flatten.columns.to_flat_index()

Index([       ('departamento', ''),           ('provincia', ''),
                       ('ano', ''),              ('semana', ''),
        ('meanAge', 'SIN_SEÑALES'),       ('meanAge', 'ALARMA'),
              ('meanAge', 'GRAVE'), ('sumCases', 'SIN_SEÑALES'),
            ('sumCases', 'ALARMA'),       ('sumCases', 'GRAVE')],
      dtype='object')

In [187]:
["_".join(a) if a[1]!='' else a[0] for a in ByProvinceWeek_sum_mean_Wide_flatten.columns.to_flat_index()]

['departamento',
 'provincia',
 'ano',
 'semana',
 'meanAge_SIN_SEÑALES',
 'meanAge_ALARMA',
 'meanAge_GRAVE',
 'sumCases_SIN_SEÑALES',
 'sumCases_ALARMA',
 'sumCases_GRAVE']

In [188]:
ByProvinceWeek_sum_mean_Wide_flatten.columns=["_".join(a) if a[1]!='' else a[0] for a in ByProvinceWeek_sum_mean_Wide_flatten.columns.to_flat_index()]
ByProvinceWeek_sum_mean_Wide_flatten

,departamento,provincia,ano,semana,meanAge_SIN_SEÑALES,meanAge_ALARMA,meanAge_GRAVE,sumCases_SIN_SEÑALES,sumCases_ALARMA,sumCases_GRAVE
0,AMAZONAS,BAGUA,2000,18,34.352941,NaN,NaN,17.0,NaN,NaN
1,AMAZONAS,BAGUA,2000,19,29.825000,NaN,NaN,40.0,NaN,NaN
2,AMAZONAS,BAGUA,2000,20,35.172414,NaN,NaN,58.0,NaN,NaN
3,AMAZONAS,BAGUA,2000,21,37.444444,NaN,NaN,27.0,NaN,NaN
4,AMAZONAS,BAGUA,2000,22,30.708333,NaN,NaN,24.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
27198,UCAYALI,PADRE ABAD,2022,51,16.111111,17.0,NaN,9.0,1.0,NaN
27199,UCAYALI,PADRE ABAD,2022,52,30.800000,33.5,NaN,5.0,2.0,NaN
27200,UCAYALI,PURUS,2020,51,46.000000,NaN,NaN,1.0,NaN,NaN
27201,UCAYALI,PURUS,2022,28,NaN,17.0,NaN,NaN,1.0,NaN


In [189]:
ByProvinceWeek_sum_mean_Wide_flatten.to_csv('dataFiles/ByProvinceWeek_sum_mean_Wide_flatten.csv',index=False)

### Wide to Long

We should be able to transfor this wide version into a long one:

In [204]:
ByProvinceWeek_sum_mean_Wide.stack()

meanAge  sumCases
departamento provincia  ano  semana enfermedad                      
AMAZONAS     BAGUA      2000 18     SIN_SEÑALES  34.352941      17.0
                             19     SIN_SEÑALES  29.825000      40.0
                             20     SIN_SEÑALES  35.172414      58.0
                             21     SIN_SEÑALES  37.444444      27.0
                             22     SIN_SEÑALES  30.708333      24.0
...                                                    ...       ...
UCAYALI      PADRE ABAD 2022 52     SIN_SEÑALES  30.800000       5.0
                                    ALARMA       33.500000       2.0
             PURUS      2020 51     SIN_SEÑALES  46.000000       1.0
                        2022 28     ALARMA       17.000000       1.0
                             51     SIN_SEÑALES  43.000000       1.0

[36422 rows x 2 columns]

In [196]:
ByProvinceWeek_sum_mean_Long=ByProvinceWeek_sum_mean_Wide_flatten.set_index(['departamento','provincia','ano','semana']).stack().reset_index()
ByProvinceWeek_sum_mean_Long

,departamento,provincia,ano,semana,level_4,0
0,AMAZONAS,BAGUA,2000,18,meanAge_SIN_SEÑALES,34.352941
1,AMAZONAS,BAGUA,2000,18,sumCases_SIN_SEÑALES,17.000000
2,AMAZONAS,BAGUA,2000,19,meanAge_SIN_SEÑALES,29.825000
3,AMAZONAS,BAGUA,2000,19,sumCases_SIN_SEÑALES,40.000000
4,AMAZONAS,BAGUA,2000,20,meanAge_SIN_SEÑALES,35.172414
...,...,...,...,...,...,...
72839,UCAYALI,PURUS,2020,51,sumCases_SIN_SEÑALES,1.000000
72840,UCAYALI,PURUS,2022,28,meanAge_ALARMA,17.000000
72841,UCAYALI,PURUS,2022,28,sumCases_ALARMA,1.000000
72842,UCAYALI,PURUS,2022,51,meanAge_SIN_SEÑALES,43.000000


In [197]:
ByProvinceWeek_sum_mean_Long.rename(columns={'level_4':'variable',0:'value'},inplace=True)
ByProvinceWeek_sum_mean_Long

,departamento,provincia,ano,semana,variable,value
0,AMAZONAS,BAGUA,2000,18,meanAge_SIN_SEÑALES,34.352941
1,AMAZONAS,BAGUA,2000,18,sumCases_SIN_SEÑALES,17.000000
2,AMAZONAS,BAGUA,2000,19,meanAge_SIN_SEÑALES,29.825000
3,AMAZONAS,BAGUA,2000,19,sumCases_SIN_SEÑALES,40.000000
4,AMAZONAS,BAGUA,2000,20,meanAge_SIN_SEÑALES,35.172414
...,...,...,...,...,...,...
72839,UCAYALI,PURUS,2020,51,sumCases_SIN_SEÑALES,1.000000
72840,UCAYALI,PURUS,2022,28,meanAge_ALARMA,17.000000
72841,UCAYALI,PURUS,2022,28,sumCases_ALARMA,1.000000
72842,UCAYALI,PURUS,2022,51,meanAge_SIN_SEÑALES,43.000000


In [198]:
ByProvinceWeek_sum_mean_Long.info() # enfermedad is object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72844 entries, 0 to 72843
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   departamento  72844 non-null  object 
 1   provincia     72844 non-null  object 
 2   ano           72844 non-null  int64  
 3   semana        72844 non-null  int64  
 4   variable      72844 non-null  object 
 5   value         72844 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.3+ MB


In [199]:
ByProvinceWeek_sum_mean_Long.variable.dtype

dtype('O')

In [206]:
ByProvinceWeek_sum_mean_Wide.stack().reset_index().enfermedad.dtype

CategoricalDtype(categories=['SIN_SEÑALES', 'ALARMA', 'GRAVE'], ordered=True, categories_dtype=object)